In [ ]:
# Make Jupyter Notebook full screen 
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
import geopandas
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.lines import Line2D

In [ ]:
DATA_PATH = '/Users/megantabbutt/BRT_data/GeoData/'
DATA_PATH_Dems = '/Users/megantabbutt/BRT_data/DemographicData/'

RoutesShapeFile = "mmt_gtfs/Metro_Transit_Bus_Routes.shp"
CitiesShapeFile = "Shapes/City/City_Limit.shp"
LakesShapeFile = "Shapes/Lakes/Lakes_and_Rivers.shp"
StreetsShapeFile = "Shapes/Street/Street_Centerlines_and_Pavement_Data.shp"


cityShapes_df = geopandas.read_file(CitiesShapeFile)
lakeShapes_df = geopandas.read_file(LakesShapeFile)
streetShapes_df = geopandas.read_file(StreetsShapeFile)

In [ ]:
df = geopandas.read_file(DATA_PATH+"tl_2018_55_bg.shp")

In [ ]:
df_dane = df[df['COUNTYFP'] == "025"]
df_dane.head(3)

In [ ]:
df_mad = pd.read_csv(DATA_PATH_Dems + "timeToWork_race_2018.txt", sep='\t')
df_mad.info()
df_mad.head(3)

In [ ]:
geoIDs_Master = [int(i) for i in list(df_dane['GEOID'])]
type(geoIDs_Master[0])

In [ ]:
densityWhite = dict()
for ID in geoIDs_Master:
    totalPeople = df_mad.loc[df_mad['GEOID'] == ID, 'Estimate!!Total'].item()
    whitePeople = df_mad.loc[df_mad['GEOID'] == ID, 'Estimate!!Total!!Population of one race!!White'].item()
    #print(totalPeople, whitePeople)
    if totalPeople > 0:
        number = whitePeople/totalPeople        
    else:
        number = None
    densityWhite[ID] = number
    
densityWhite

In [ ]:
densityBlack = dict()
for ID in geoIDs_Master:
    totalPeople = df_mad.loc[df_mad['GEOID'] == ID, 'Estimate!!Total'].item()
    blackPeople = df_mad.loc[df_mad['GEOID'] == ID, 'Estimate!!Total!!Population of one race!!Black or African American'].item()
    #print(totalPeople, whitePeople)
    if totalPeople > 0:
        number = blackPeople/totalPeople        
    else:
        number = None
    densityBlack[ID] = number
    
densityBlack

In [ ]:
df_mad_Hisp = pd.read_csv(DATA_PATH_Dems + "hispanicPopulation_2018.txt", sep='\t')
df_mad_Hisp.info()
df_mad_Hisp.head(3)

In [ ]:
densityHisp = dict()
for ID in geoIDs_Master:
    totalPeople = df_mad_Hisp.loc[df_mad_Hisp['GEOID'] == ID, 'Total'].item()
    HipPeople = df_mad_Hisp.loc[df_mad['GEOID'] == ID, 'totalHispanic'].item()
    #print(totalPeople, HipPeople)
    if totalPeople > 0:
        number = HipPeople/totalPeople        
    else:
        number = None
    densityHisp[ID] = number
    
densityHisp

In [ ]:
densityHispBlack = dict()
for ID in geoIDs_Master:
    totalPeople = df_mad_Hisp.loc[df_mad_Hisp['GEOID'] == ID, 'Total'].item()
    HispBlackPeople = (df_mad_Hisp.loc[df_mad['GEOID'] == ID, 'totalHispanic'].item() + 
                       df_mad.loc[df_mad['GEOID'] == ID, 'Estimate!!Total!!Population of one race!!Black or African American'].item())
    #print(totalPeople, HipPeople)
    if totalPeople > 0:
        number = HispBlackPeople/totalPeople        
    else:
        number = None
    densityHispBlack[ID] = number
    
densityHispBlack

In [ ]:
df_dane['densityWhite'] = list(densityWhite.values())
df_dane['densityBlack'] = list(densityBlack.values())
df_dane['densityHisp'] = list(densityHisp.values())
df_dane['densityHispBlack'] = list(densityHispBlack.values())
df_dane.head(3)

In [ ]:
route_freq_wkday = {2: {5: 1, 6: 2, 7: 2, 8: 2, 9: 2, 10: 2, 11: 2, 12: 2, 13: 2, 14: 2, 15: 2, 16: 2, 17: 2, 18: 2, 19: 2, 20: 2, 21: 2, 22: 2, 23: 1}, 
                    4: {5: 1, 6: 2, 7: 2, 8: 2, 9: 2, 10: 2, 11: 2, 12: 2, 13: 2, 14: 2, 15: 2, 16: 2, 17: 2, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1}, 
                    5: {5: 1, 6: 2, 7: 2, 8: 2, 9: 2, 10: 2, 11: 2, 12: 2, 13: 2, 14: 2, 15: 2, 16: 2, 17: 2, 18: 2, 19: 1, 20: 1, 21: 1, 22: 1, 23: 1}, 
                    6: {5: 1, 6: 2, 7: 2, 8: 2, 9: 2, 10: 2, 11: 2, 12: 2, 13: 2, 14: 2, 15: 2, 16: 2, 17: 2, 18: 2, 19: 2, 20: 2, 21: 2, 22: 3, 23: 1}, 
                    7: {5: 1, 6: 2, 7: 2, 8: 2, 9: 2, 10: 2, 11: 2, 12: 2, 13: 2, 14: 2, 15: 2, 16: 2, 17: 2, 18: 2, 19: 2, 20: 2, 21: 2, 22: 2}, 
                    8: {7: 2, 8: 2, 9: 2, 10: 2, 11: 2, 12: 2, 13: 2, 14: 2, 15: 2, 16: 2, 17: 2, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1}, 
                    10: {6: 1, 7: 4, 8: 4, 9: 4, 10: 4, 11: 4, 12: 4, 13: 4, 14: 4, 15: 4, 16: 4, 17: 4, 18: 2}, 
                    11: {6: 1, 7: 1, 8: 1, 9: 1, 15: 2, 16: 1, 17: 1}, 
                    12: {6: 1, 7: 1, 8: 1, 9: 1, 14: 1, 15: 1, 16: 1, 17: 1}, 
                    13: {5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1, 23: 1}, 
                    15: {6: 1, 7: 2, 8: 2, 9: 2, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 2, 16: 2, 17: 2, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1}, 
                    16: {5: 1, 6: 3, 7: 2, 8: 2, 9: 2, 10: 2, 11: 2, 12: 2, 13: 2, 14: 2, 15: 2, 16: 2, 17: 2, 18: 1, 19: 1, 20: 1, 21: 1, 22: 2}, 
                    17: {6: 1, 7: 2, 8: 2, 9: 2, 10: 2, 11: 2, 12: 2, 13: 2, 14: 2, 15: 2, 16: 2, 17: 2, 18: 2, 19: 2, 20: 2, 21: 2, 22: 2}, 
                    18: {5: 1, 6: 2, 7: 2, 8: 2, 9: 2, 10: 2, 11: 2, 12: 2, 13: 2, 14: 2, 15: 2, 16: 2, 17: 2, 18: 2, 19: 2, 20: 2, 21: 2, 22: 2, 23: 1}, 
                    20: {6: 2, 7: 2, 8: 2, 9: 2, 10: 2, 11: 2, 12: 2, 13: 2, 14: 2, 15: 2, 16: 2, 17: 2, 18: 2, 19: 2, 20: 2, 21: 2, 22: 1}, 
                    21: {5: 1, 6: 2, 7: 2, 8: 2, 9: 2, 10: 2, 11: 2, 12: 2, 13: 2, 14: 2, 15: 2, 16: 2, 17: 2, 18: 1, 19: 1, 20: 1, 21: 1, 22: 2}, 
                    22: {5: 2, 6: 2, 7: 2, 8: 2, 9: 2, 10: 2, 11: 2, 12: 2, 13: 2, 14: 2, 15: 2, 16: 2, 17: 2, 18: 2, 19: 1, 20: 1, 21: 1, 22: 1}, 
                    23: {6: 1, 7: 2, 8: 1, 16: 2, 17: 2}, 
                    26: {6: 1, 7: 2, 8: 2, 9: 2, 10: 2, 11: 2, 12: 2, 13: 2, 14: 2, 15: 2, 16: 2, 17: 2, 18: 2, 19: 2, 20: 2, 21: 2, 22: 2}, 
                    27: {6: 1, 7: 2, 8: 1, 15: 1, 16: 2, 17: 2, 18: 1}, 
                    30: {5: 1, 6: 2, 7: 2, 8: 2, 9: 2, 10: 2, 11: 2, 12: 2, 13: 2, 14: 2, 15: 2, 16: 2, 17: 2, 18: 2, 19: 2, 20: 2, 21: 2, 22: 1}, 
                    31: {6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1}, 
                    32: {6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1}, 
                    33: {5: 1, 6: 2, 7: 2, 8: 1, 15: 1, 16: 2, 17: 2, 18: 1}, 
                    36: {7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1}, 
                    39: {6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16: 1, 17: 1}, 
                    40: {5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1}, 
                    44: {6: 2, 7: 2, 8: 2, 9: 1, 14: 1, 15: 1, 16: 2, 17: 2, 18: 1}, 
                    48: {6: 1, 7: 2, 16: 1, 17: 2}, 
                    49: {6: 2, 7: 2, 8: 1, 16: 2, 17: 2, 18: 1}, 
                    50: {5: 1, 6: 2, 7: 2, 8: 2, 9: 2, 10: 2, 11: 2, 12: 2, 13: 2, 14: 2, 15: 2, 16: 2, 17: 2, 18: 2, 19: 2, 20: 2, 21: 2, 22: 2, 23: 1}, 
                    51: {5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1}, 
                    52: {6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1}, 
                    55: {6: 1, 7: 1, 8: 1, 16: 1, 17: 2, 18: 1}, 
                    67: {5: 1, 6: 2, 7: 2, 8: 2, 9: 2, 10: 2, 11: 2, 12: 2, 13: 2, 14: 2, 15: 2, 16: 2, 17: 2, 18: 2, 19: 2, 20: 2, 21: 2, 22: 2}, 
                    70: {5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1}, 
                    72: {6: 1, 7: 2, 8: 1, 15: 2, 16: 2, 17: 2, 18: 1}, 
                    73: {5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1, 23: 1}, 
                    75: {6: 2, 7: 1, 8: 2, 16: 2, 17: 1, 18: 2}, 
                    81: {0: 2, 1: 2, 18: 1, 19: 2, 20: 2, 21: 2, 22: 2, 23: 2}, 
                    82: {0: 2, 1: 2, 2: 1, 19: 2, 20: 2, 21: 2, 22: 2, 23: 2}, 
                    84: {9: 4, 10: 4, 11: 4, 12: 4, 13: 4, 14: 4, 15: 4, 16: 5, 17: 4, 18: 4}}

In [ ]:
route_freq_wkday[2][6]

In [ ]:
RoutesShapeFile = "mmt_gtfs/Metro_Transit_Bus_Routes.shp"
routesShapes_df = geopandas.read_file(RoutesShapeFile)
routesShapes_df.info()
routesShapes_df.head(3)

In [ ]:
def route_freq_time(time24):
    frequency = []
    for i in list(routesShapes_df['route_shor']):
        if int(i) in route_freq_wkday:
            if time24 in route_freq_wkday[int(i)]:
                frequency.append(route_freq_wkday[int(i)][time24])
            else: 
                frequency.append(0)
        else:
            frequency.append(0)
    
    if len(frequency) == 47:   
        return frequency

print(route_freq_time(6))

In [ ]:
routesShapes_df['frequency_1am'] = route_freq_time(1)
routesShapes_df['frequency_2am'] = route_freq_time(2)
routesShapes_df['frequency_3am'] = route_freq_time(3)
routesShapes_df['frequency_4am'] = route_freq_time(4)
routesShapes_df['frequency_5am'] = route_freq_time(5)
routesShapes_df['frequency_6am'] = route_freq_time(6)
routesShapes_df['frequency_7am'] = route_freq_time(7)
routesShapes_df['frequency_8am'] = route_freq_time(8)
routesShapes_df['frequency_9am'] = route_freq_time(9)
routesShapes_df['frequency_10am'] = route_freq_time(10)
routesShapes_df['frequency_11am'] = route_freq_time(11)
routesShapes_df['frequency_12pm'] = route_freq_time(12)
routesShapes_df['frequency_1pm'] = route_freq_time(13)
routesShapes_df['frequency_2pm'] = route_freq_time(14)
routesShapes_df['frequency_3pm'] = route_freq_time(15)
routesShapes_df['frequency_4pm'] = route_freq_time(16)
routesShapes_df['frequency_5pm'] = route_freq_time(17)
routesShapes_df['frequency_6pm'] = route_freq_time(18)
routesShapes_df['frequency_7pm'] = route_freq_time(19)
routesShapes_df['frequency_8pm'] = route_freq_time(20)
routesShapes_df['frequency_9pm'] = route_freq_time(21)
routesShapes_df['frequency_10pm'] = route_freq_time(22)
routesShapes_df['frequency_11pm'] = route_freq_time(23)
routesShapes_df['frequency_12am'] = route_freq_time(0)
routesShapes_df.head(3)

In [ ]:
suptitle = "Distribution of White Population and 7am Routes"
savefigName = 'PaperFigures/Demographics7am'
colName1 = 'frequency_7am'
colName2 = 'frequency_7am'

fig, ax = plt.subplots(1, 1, figsize=(16, 16))
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=-1.5)

df_dane.plot(column='densityWhite', ax=ax, legend=True, legend_kwds={'label': "Percentage White"}, cax=cax, cmap='magma', alpha=.8)

lakeShapes_df.plot(color='lightblue', alpha=.9, ax=ax)
streetShapes_df.plot(color='#fbf7f4', alpha=.1, ax=ax)

#for i in route_freq_wkday:
    #for j in range(routesShapes_df.loc[routesShapes_df['route_shor']==i, colName2].item()):
        #routesShapes_df[routesShapes_df['route_shor']==i].plot(ax=ax, linestyle="--", alpha=1, linewidth=1.5, color='#be0119')
        
for i in route_freq_wkday:
    for j in range(routesShapes_df.loc[routesShapes_df['route_shor']==i, colName1].item()):
        routesShapes_df[routesShapes_df['route_shor']==i].plot(ax=ax, linestyle="-", alpha=1, linewidth=1.5, color='#380835')
        
        
custom_lines = [Line2D([0], [0], color='#380835', lw=4)]
ax.legend(custom_lines, ['7am Routes'], loc='upper left', framealpha=1)

cax.set_ylabel("Percentage White", fontsize=16)

ax.set_facecolor("#e7e3e0")
ax.set_xlim(-89.6, -89.2)
ax.set_ylim(42.97, 43.2)
ax.set_yticks([])
ax.set_xticks([])

fig.suptitle(suptitle, fontsize=18, x=.55, y=.76)
plt.savefig('PaperFigures/White7am', bbox_inches='tight')
plt.savefig('PaperFigures/White7am', bbox_inches='tight', format='svg')
plt.show()

In [ ]:
suptitle = "Distribution of Hispanic/Black Population and 2pm Routes"
colName1 = 'frequency_7am'
colName2 = 'frequency_2pm'

fig, ax = plt.subplots(1, 1, figsize=(16, 16))
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=-1.5)

df_dane.plot(column='densityHispBlack', ax=ax, legend=True, cax=cax, cmap='magma_r', alpha=.8)

lakeShapes_df.plot(color='lightblue', alpha=.9, ax=ax)
streetShapes_df.plot(color='#fbf7f4', alpha=.1, ax=ax)

for i in route_freq_wkday:
    for j in range(routesShapes_df.loc[routesShapes_df['route_shor']==i, colName1].item()):
        routesShapes_df[routesShapes_df['route_shor']==i].plot(ax=ax, linestyle="--", alpha=1, linewidth=1.5, color='#4baa82')
        
for i in route_freq_wkday:
    for j in range(routesShapes_df.loc[routesShapes_df['route_shor']==i, colName2].item()):
        routesShapes_df[routesShapes_df['route_shor']==i].plot(ax=ax, linestyle="-", alpha=1, linewidth=1.5, color='#380835')
        
        
custom_lines = [Line2D([0], [0], color='#4baa82', lw=4), Line2D([0], [0], color='#380835', lw=4)]
ax.legend(custom_lines, ['7am Routes', '2pm Routes'], loc='upper left', framealpha=1)

cax.set_ylabel("Percentage Hispanic and Black", fontsize=16)

ax.set_facecolor("#e7e3e0")
ax.set_xlim(-89.6, -89.2)
ax.set_ylim(42.97, 43.2)
ax.set_yticks([])
ax.set_xticks([])

fig.suptitle(suptitle, fontsize=18, x=.55, y=.76)
plt.savefig('PaperFigures/HispBlack2pm', bbox_inches='tight')
plt.savefig('PaperFigures/HispBlack2pm', bbox_inches='tight', format='svg')
plt.show()

In [ ]:
suptitle = "Distribution of Black/African American Population and 10pm Routes"
colName1 = 'frequency_7am'
colName2 = 'frequency_10pm'

fig, ax = plt.subplots(1, 1, figsize=(16, 16))
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=-1.5)

df_dane.plot(column='densityBlack', ax=ax, legend=True, cax=cax, cmap='magma_r', alpha=.8)

lakeShapes_df.plot(color='lightblue', alpha=.9, ax=ax)
streetShapes_df.plot(color='#fbf7f4', alpha=.1, ax=ax)

for i in route_freq_wkday:
    for j in range(routesShapes_df.loc[routesShapes_df['route_shor']==i, colName1].item()):
        routesShapes_df[routesShapes_df['route_shor']==i].plot(ax=ax, linestyle="--", alpha=1, linewidth=1.5, color='#4baa82')
        
for i in route_freq_wkday:
    for j in range(routesShapes_df.loc[routesShapes_df['route_shor']==i, colName2].item()):
        routesShapes_df[routesShapes_df['route_shor']==i].plot(ax=ax, linestyle="-", alpha=1, linewidth=1.5, color='#380835')
        
        
custom_lines = [Line2D([0], [0], color='#4baa82', lw=4), Line2D([0], [0], color='#380835', lw=4)]
ax.legend(custom_lines, ['7am Routes', '10pm Routes'], loc='upper left', framealpha=1)

cax.set_ylabel("Percentage Black/African American", fontsize=16)

ax.set_facecolor("#e7e3e0")
ax.set_xlim(-89.6, -89.2)
ax.set_ylim(42.97, 43.2)
ax.set_yticks([])
ax.set_xticks([])

fig.suptitle(suptitle, fontsize=18, x=.55, y=.76)
plt.savefig('PaperFigures/Black10pm', bbox_inches='tight')
plt.savefig('PaperFigures/Black10pm', bbox_inches='tight', format='svg')
plt.show()

In [ ]:
suptitle = "Distribution of Hispanic/Latino Population and 2am Routes"
colName1 = 'frequency_7am'
colName2 = 'frequency_2am'

fig, ax = plt.subplots(1, 1, figsize=(16, 16))
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=-1.5)

df_dane.plot(column='densityHispBlack', ax=ax, legend=True, cax=cax, cmap='magma_r', alpha=.8)

lakeShapes_df.plot(color='lightblue', alpha=.9, ax=ax)
streetShapes_df.plot(color='#fbf7f4', alpha=.1, ax=ax)

for i in route_freq_wkday:
    for j in range(routesShapes_df.loc[routesShapes_df['route_shor']==i, colName1].item()):
        routesShapes_df[routesShapes_df['route_shor']==i].plot(ax=ax, linestyle="--", alpha=1, linewidth=1.5, color='#4baa82')
        
for i in route_freq_wkday:
    for j in range(routesShapes_df.loc[routesShapes_df['route_shor']==i, colName2].item()):
        routesShapes_df[routesShapes_df['route_shor']==i].plot(ax=ax, linestyle="-", alpha=1, linewidth=1.5, color='#380835')
        
        
custom_lines = [Line2D([0], [0], color='#4baa82', lw=4), Line2D([0], [0], color='#380835', lw=4)]
ax.legend(custom_lines, ['7am Routes', '2am Routes'], loc='upper left', framealpha=1)

cax.set_ylabel("Percentage Hispanic/Latino", fontsize=16)

ax.set_facecolor("#e7e3e0")
ax.set_xlim(-89.6, -89.2)
ax.set_ylim(42.97, 43.2)
ax.set_yticks([])
ax.set_xticks([])

fig.suptitle(suptitle, fontsize=18, x=.55, y=.76)
plt.savefig('PaperFigures/Hisp2am', bbox_inches='tight')
plt.savefig('PaperFigures/Hisp2am', bbox_inches='tight', format='svg')
plt.show()

In [ ]:
#suptitle = "Cars per Capita vs Route Frequency - Weekday, 2am"
#savefigName = 'PaperFigures/CarOwnershipVSfrequency2am'
colName1 = 'frequency_2pm'
colName2 = 'frequency_7am'

fig, ax = plt.subplots(1, 1, figsize=(16, 16))
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=-1.5)

df_dane.plot(column='densityBlack', ax=ax, legend=True, legend_kwds={'label': "Percentage Black"}, cax=cax, cmap='magma_r', alpha=.8)

lakeShapes_df.plot(color='lightblue', alpha=.9, ax=ax)
streetShapes_df.plot(color='#fbf7f4', alpha=.1, ax=ax)

for i in route_freq_wkday:
    for j in range(routesShapes_df.loc[routesShapes_df['route_shor']==i, colName2].item()):
        routesShapes_df[routesShapes_df['route_shor']==i].plot(ax=ax, linestyle="--", alpha=1, linewidth=1.5, color='#be0119')
        
for i in route_freq_wkday:
    for j in range(routesShapes_df.loc[routesShapes_df['route_shor']==i, colName1].item()):
        routesShapes_df[routesShapes_df['route_shor']==i].plot(ax=ax, linestyle="-", alpha=1, linewidth=1.5, color='#380835')
        
        
custom_lines = [Line2D([0], [0], color='#be0119', lw=4), Line2D([0], [0], color='#380835', lw=4)]
ax.legend(custom_lines, ['7am Routes', '2pm Routes'], loc='upper left', framealpha=1)

ax.set_facecolor("#e7e3e0")
ax.set_xlim(-89.6, -89.2)
ax.set_ylim(42.97, 43.2)
ax.set_yticks([])
ax.set_xticks([])

fig.suptitle("7am vs 2pm Route Coverage vs Percentage Black", fontsize=18, x=.55, y=.76)
#plt.savefig(savefigName, bbox_inches='tight')
plt.show()